In [8]:

import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime
import openpyxl

In [9]:
rd = pd.read_excel('PowerBI Monthly Sales Dashboard FY2021-22_sc.xlsx', sheet_name='Raw Data')

In [10]:
rd.columns = [' '.join(i.split()) for i in rd.columns]
rd.columns = rd.columns.str.lower()
rd.columns = rd.columns.str.replace(' ','_')
rd1 = rd[['date','month','country','r._dealer_name','sub_category','product_code','product_packaging',\
          'sales_after_discount','data_type','focus_type','fiscal_year']]
rd1.rename(columns={'r._dealer_name':'dealer_name', 'sales_after_discount': 'sales'}, inplace=True)
fy21 = rd1[rd1.fiscal_year=='2021/2022']
fy21.dealer_name = fy21.dealer_name.astype(str)
fy21.dealer_name = [' '.join(i.split()) for i in fy21.dealer_name]
fy21.product_code=fy21.product_code.astype(str)
fy21.product_code = [' '.join(i.split()) for i in fy21.product_code]

In [11]:

def powerbi(cty):
    fy21_c = fy21[fy21.country==cty]
    
    def cat(x,y,z):

        for m, s in zip(y,z):
            if m == 'Apr':
                df.loc[0,x]=s
            elif m == 'May':
                df.loc[1,x]=s
            elif m == 'Jun':
                df.loc[2,x]=s
            elif m == 'Jul':
                df.loc[3,x]=s
            elif m == 'Aug':
                df.loc[4,x]=s
            elif m == 'Sep':
                df.loc[5,x]=s
            elif m == 'Oct':
                df.loc[6,x]=s
            elif m == 'Nov':
                df.loc[7,x]=s
            elif m == 'Dec':
                df.loc[8,x]=s
            elif m == 'Jan':
                df.loc[9,x]=s
            elif m == 'Feb':
                df.loc[10,x]=s
            elif m == 'Mar':
                df.loc[11,x]=s
        return df.round(0)

    d = pd.date_range(start='2021-Apr', end='2022-04', freq='m')
    df = pd.DataFrame({'month':d})
    df.month = df.month.dt.to_period('m')

    # work on budget
    bud = fy21_c[fy21_c.data_type=='Budget']
    bud_s = bud[bud.focus_type=='Speed']
    bud_s_group = bud_s.groupby('month')['sales'].sum().reset_index()

    x = 'speed_t'
    y= bud_s_group.month
    z = bud_s_group.sales
    df = cat(x,y,z)


    # add growth into the dataframe df
    bud_g = bud[bud.focus_type=='Growth']
    bud_g_group = bud_g.groupby('month')['sales'].sum().reset_index()

    # integrate growth into df
    x = 'growth_t'
    y= bud_g_group.month
    z = bud_g_group.sales
    df = cat(x,y,z)

    # create new total column
    df['total_t']=df.speed_t + df.growth_t
    df['total_t_c']=df.total_t.cumsum()

    # cumsum for speed and growth target
    df['speed_t_c']=df.speed_t.cumsum()
    df['growth_t_c']=df.growth_t.cumsum()

    # add actual into df
    act = fy21_c[fy21_c.data_type=='Actual']

    # group speed sales by month
    act_s = act[act.focus_type=='Speed']
    act_s_group = act_s.groupby('month')['sales'].sum().reset_index()

    # integrate act speed sales into df
    x = 'speed_a'
    y= act_s_group.month
    z = act_s_group.sales
    df = cat(x,y,z)

    # group growth sales by month
    act_g = act[act.focus_type=='Growth']
    act_g_group = act_g.groupby('month')['sales'].sum().reset_index()

    # integrate act growth sales into df
    x = 'growth_a'
    y= act_g_group.month
    z = act_g_group.sales
    df = cat(x,y,z)

    # cumsum for speed and growth actual
    df['speed_a_c']=df.speed_a.cumsum()
    df['growth_a_c']=df.growth_a.cumsum()

    # add act
    df['total_a']=df.speed_a + df.growth_a
    df['total_a_c']=df.total_a.cumsum()
    df['country']=cty

    # groupby dealer
    dd = fy21_c[['dealer_name','data_type','sales']]
    ddg = dd.groupby(['dealer_name','data_type'])['sales'].sum().reset_index()
    ddg['country']=cty

    return df, ddg

In [12]:
wr = pd.DataFrame()
ab = pd.DataFrame()
cty_list = ['Malaysia','Thailand','Philippines','Singapore','Vietnam','Indonesia']

for cty in cty_list:
    a, b = powerbi(cty)
    wr = wr.append(a)
    ab = ab.append(b)

In [13]:
wr.to_csv('pbi_weekly_report_all.csv', index=False)

In [14]:
ab.to_csv('pbi_dealer_actual_budget_all.csv', index=False)